# Using DeepEval Metrics with Experiments 🔍

In this notebook, we'll explore how to integrate [DeepEval](https://github.com/confident-ai/deepeval), a popular LLM evaluation framework, with Datadog LLM Observability experiments.

DeepEval provides sophisticated evaluation metrics like:
- **Answer Relevancy**: Measures if the output is relevant to the input question
- **G-Eval**: Customizable LLM-as-a-judge evaluations for any criteria
- **Faithfulness**: Ensures outputs stay true to provided context (useful for RAG applications)
- And many more!

By integrating DeepEval with Datadog experiments, you can combine advanced semantic evaluations with Datadog's experiment tracking and visualization capabilities.

## 0. Set up

First, let's import the required libraries and initialize both Datadog LLM Observability and DeepEval.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file.
load_dotenv(override=True)

from typing import Dict, Any
from ddtrace.llmobs import LLMObs
from openai import OpenAI

# Import DeepEval components
from deepeval.metrics import AnswerRelevancyMetric, GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams


LLMObs.enable(
    api_key=os.getenv("DD_API_KEY"),
    app_key=os.getenv("DD_APPLICATION_KEY"),
    project_name="Onboarding",
    ml_app="Onboarding-ML-App")

oai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 1. Understanding DeepEval Basics

Before integrating with Datadog experiments, let's see how DeepEval metrics work.

It is possible to use DeepEval metrics in the following way:
1. Create a metric instance with a threshold
2. Create an `LLMTestCase` with your inputs and outputs
3. Call `metric.measure(test_case)` to evaluate
4. Access `metric.score` to get the result

In [ ]:
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    # Replace this with the actual output from your LLM application
    actual_output="We offer a 30-day full refund at no extra costs.",
    retrieval_context=["All customers are eligible for a 30 day full refund at no extra costs."]
)

In [ ]:
answer_relevancy_metric.measure(test_case)
display(answer_relevancy_metric.score)
# All metrics also offer an explanation
display(answer_relevancy_metric.reason)

The metric returns a score (typically between 0 and 1) and a `reason` explaining the evaluation. This makes it easy to understand why a particular score was assigned.

## 2. Integrating DeepEval with Datadog Experiments

Now that we understand how DeepEval works, let's integrate it with Datadog experiments.

**The Challenge**: Datadog evaluators receive `input_data`, `output_data`, and `expected_output` as separate parameters, but DeepEval metrics expect a single `LLMTestCase` object.

**The Solution**: Create evaluator functions that convert Datadog's format into DeepEval's `LLMTestCase` format, then measure and return the score.

In [ ]:
# Pull the dataset from Datadog
dataset = LLMObs.pull_dataset(dataset_name="capitals-of-the-world")
dataset.as_dataframe()

For this example, we'll reuse the `capitals-of-the-world` dataset from the previous notebooks.

In [ ]:
# Create DeepEval metric instances
# These can be reused across multiple evaluations
EVAL_MODEL = "gpt-4o-mini"
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7, model=EVAL_MODEL)

correctness_geval_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=EVAL_MODEL
)


# Evaluator functions that bridge Datadog and DeepEval
def answer_relevancy_evaluator(input_data, output_data, expected_output):
    """Evaluates if the output is relevant to the input question."""
    test_case = LLMTestCase(
        input=input_data.get("question", ""),
        actual_output=output_data
    )
    answer_relevancy_metric.measure(test_case)
    score = answer_relevancy_metric.score
    return score


def correctness_geval_evaluator(input_data, output_data, expected_output):
    """Custom G-Eval evaluator for correctness using LLM-as-judge."""
    test_case = LLMTestCase(
        input=input_data.get("question", ""),
        actual_output=output_data,
        expected_output=expected_output
    )
    correctness_geval_metric.measure(test_case)
    score = correctness_geval_metric.score
    return score


display("✅ DeepEval evaluators created!")

## 3. Running an Experiment with DeepEval Metrics

Now let's create an experiment that combines traditional evaluators (exact match, substring check) with DeepEval's advanced semantic evaluators.

We'll use the same capital cities dataset and task from the previous notebooks, but add DeepEval metrics.

In [ ]:
# Task function: Generate capital city names
def generate_capital(input_data: Dict[str, Any], config: Dict[str, Any]) -> str:
    output = oai_client.chat.completions.create(
        model=config["model"],
        messages=[{"role": "user", "content": input_data["question"]}],
        temperature=config["temperature"]
    )
    return output.choices[0].message.content


# Traditional evaluators for comparison
def exact_match(input_data, output_data, expected_output):
    return expected_output == output_data


def contains_answer(input_data, output_data, expected_output):
    return expected_output in output_data


# Define the experiment combining traditional and DeepEval evaluators
experiment = LLMObs.experiment(
    name="deepeval-integration-example",
    dataset=dataset,
    task=generate_capital,
    evaluators=[
        exact_match,                    # Traditional: Exact string match
        contains_answer,                # Traditional: Substring check
        answer_relevancy_evaluator,     # DeepEval: Semantic relevancy
        correctness_geval_evaluator     # DeepEval: LLM-as-judge correctness
    ],
    config={"model": "gpt-4o-mini", "temperature": 0},
    description="Experiment combining traditional evaluators with DeepEval metrics",
)

In [ ]:
results = experiment.run(jobs=5)

Let's run the experiment! This will execute the task against each row in the dataset and evaluate using both traditional and DeepEval metrics.

In [ ]:
experiment.url

In Datadog, you'll see all evaluator results side-by-side:
- **Traditional evaluators** (`exact_match`, `contains_answer`) provide binary pass/fail results
- **DeepEval evaluators** (`answer_relevancy_evaluator`, `correctness_geval_evaluator`) provide nuanced scores (0-1).

After the experiment completes, you can view it in Datadog (it may take a few seconds to be accessible).

## Awesome! 🎉

You've successfully integrated DeepEval metrics with Datadog LLM Observability experiments! By creating simple bridge functions that convert Datadog's evaluator format to DeepEval's `LLMTestCase` format, you can now use sophisticated LLM evaluation metrics like Answer Relevancy and G-Eval alongside traditional evaluators.

You can explore other DeepEval metrics like `FaithfulnessMetric` and `HallucinationMetric` for RAG applications, or create custom G-Eval criteria for domain-specific evaluations.

---

Feel free to experiment with different DeepEval metrics, adjust thresholds, and combine them with your own custom evaluators to build comprehensive evaluation pipelines.